In [1]:
#-----------------------------------------------------------------------------------------------------
# BERT와 sLLM 모델을 이용한 질의응답 서비스 구축 예
# - sLLM_test_embed.ipynb에서 임베딩된 문서들을 이용하여 프롬프트 생성 및 입력하는 예시임.
#
# 질의 응답 시스템 과정
# 문서들 전처리 : 
#    단락별루 분할(\n\n) - 불용어 제거 -문장별루 분할.
# 임베딩 : 
#    kpf-sbert-v1.1로  문장 평균 임베딩벡터 구함 - es에 문장별루 단락text와 평균벡터 저장.
# 프롬프트생성 및 입력 : 
#   검색어 입력(회사:과장일때 휴가 일수는 얼마?)-bert로 임베딩 검색(*스코어가 0.6이상인 경우 체택)-sLLM에 검색된 단락 text를 문맥으로 해서 prompt 구성
#   sLLM에 prompt 입력-응답 결과 출력
#-----------------------------------------------------------------------------------------------------
import os
import random
import numpy as np
import pandas as pd
import time
import random

import sys
sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

LOGGER = mlogging(loggername="sllm-test", logfilename='../../log/sll-test.txt') # 로그

# param-----------------------------------
SEED = 111
seed_everything(SEED)
DEVICE = GPU_info() # GPU 혹은 CPU
OUT_DIMENSION = 128   # 출력 dimension 128 혹은 0(768)
EMBEDDING_METHOD=1 # 1=평균
NUM_CLUSTERS=10
NUM_CLUSTERS_VARIABLE=False
   
BATCH_SIZE=20
FLOAT_TYPE = 'float16'

# ES 접속
ES_URL = 'http://10.10.4.10:9200/'             # elasticsearch 접속 url
ES_INDEX_NAME = 'mpower_doc_128d_1'            # elasticsearch 인덱스명
ES_INDEX_FILE = './data/mpower10u_128d_1.json' # 인덱스 생성 파일
SEARCH_SIZE = 3                                # 검색 계수
MIN_SCORE = 1.3                                # 검색 1.4 스코어 이하면 제거

# 임베딩 모델 param
MODEL_PATH = '../../data11/model/kpf-sbert-128d-v1'
POLLING_MODE = 'mean'

# sLLM 모델 param
LLM_MODEL_TYPE = 2      # 0=아래 지정한 sLLM 모델, 1=GPT 모델, 2=BARD 모델.

lora_weights:str = '../../data11/model/LLM/beomi/KoAlpaca-Polyglot-5.8B/'      # lora weight 경로
llm_model_path:str ='../../data11/model/LLM/beomi/KoAlpaca-Polyglot-5.8B/'     # llm 모델경로(KoAlpaca-Polyglot-5.8B 모델이 가장 속도가 빠르고, 잘 응답하는것 같음.)
uselora_weight = False # Lora 사용하는 경우 True
load_8bit = True       # 8bit 로딩 

# prompt_template(모델에 따라 변경)
PROMPT_DICT = {
    #"prompt_context":("아래 내용을 가지고 질문에 대해 간략히 답변해 주세요\n\n### 내용: {context}\n\n### 질문: {query}\n\n### 답변:"),
    #"prompt_no_context":("질문에 대해 간략히 답변해 주세요\n\n### 질문: {query}\n\n### 답변:")
     
    #"prompt_context":("### 질문: {query}\n\n아래 내용을 가지고 질문에 대해 간략히 답변해 주세요\n\n### 내용: {context}\n\n### 답변:"),
    #"prompt_no_context":("### 질문: {query}\n\n질문에 대해 간략히 답변해 주세요\n\n### 답변:")
    
    #"prompt_context":("### 질문: {context}\n\n위 내용을 간략히 요약해 주세요\n\n### 답변:"),
    #"prompt_no_context":("### 질문: {context}\n\n위 내용을 간략히 요약해 주세요\n\n### 답변:")
    
    #"prompt_context":("{context}\n위 내용을 간략히 요약해 주세요\n\n"),
    #"prompt_no_context":("{context}\n위 내용을 간략히 요약해 주세요\n\n"),
    
    "prompt_context":("다음 내용을 파악하여 질문에 대해 간략한 답변을 작성하세요.\n### 내용: {context}\n### 질문: {query}\n### 답변:"),
    "prompt_no_context":("질문에 대해 간략한 답변을 작성하세요.\n### 질문: {query} ### 답변:")
}
#-------------------------------------------

# prompt 테스트 
print(f'\n\nprompt 테스트----------------------\n')
query='제주도의 크기는 얼마?'
#context = ''
context = '제주도는 최남단에 있는 섬으로, 길이는 40km가 되는 대한민국에서 가장큰 섬이다.'
if context:
    prompt = PROMPT_DICT['prompt_context'].format(query=query, context=context)
else:
    prompt = PROMPT_DICT['prompt_no_context'].format(query=query)
    
print(prompt)

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


logfilepath:../../log/sll-test.txt_2023-06-23.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


prompt 테스트----------------------

다음 내용을 파악하여 질문에 대해 간략한 답변을 작성하세요.
### 내용: 제주도는 최남단에 있는 섬으로, 길이는 40km가 되는 대한민국에서 가장큰 섬이다.
### 질문: 제주도의 크기는 얼마?
### 답변:


In [2]:
#----------------------------------------------------------------------
# 임베딩 모델 로딩
#----------------------------------------------------------------------
from tqdm.notebook import tqdm
from myutils import embed_text, bi_encoder, mpower_index_batch

TOKENIZER1, BI_ENCODER1 = bi_encoder(model_path=MODEL_PATH, max_seq_len=512, do_lower_case=True, pooling_mode=POLLING_MODE, out_dimension=OUT_DIMENSION, device=DEVICE)


In [3]:
#----------------------------------------------------------------------
# sLLM 모델 로딩
#----------------------------------------------------------------------
import torch
import transformers
from peft import PeftModel
import time

if LLM_MODEL_TYPE == 0:     # 0=아래 지정한 sLLM 모델, 1=GPT 모델, 2=BARD 모델. == True:
    start_time = time.time()

    # tokenizer 로딩
    tokenizer = transformers.AutoTokenizer.from_pretrained(llm_model_path)

    # 원본 모델 로딩
    model = transformers.AutoModelForCausalLM.from_pretrained(llm_model_path, load_in_8bit=load_8bit, torch_dtype=torch.float16, device_map="auto")

    if uselora_weight:
        model = PeftModel.from_pretrained(model, lora_weights, torch_dtype=torch.float16) # loRA 모델 로딩

    if not load_8bit:
        model.half()

    model.eval()

    end_time = time.time() - start_time
    print("time: {:.2f} ms\n".format(end_time * 1000)) 
    print(model)
    


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /MOCOMSYS/anaconda3/envs/bong/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/MOCOMSYS/anaconda3/envs/bong/lib/libcudart.so.11.0'), PosixPath('/MOCOMSYS/anaconda3/envs/bong/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [4]:
# ES 관련
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk

#--------------------------------------------
# 조건에 맞게 임베딩 처리하는 함수 
#--------------------------------------------
def embedding(paragraphs:list)->list:
    # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
    embeddings = embed_text(model=BI_ENCODER1, paragraphs=paragraphs, return_tensor=False).astype(FLOAT_TYPE)    
    return embeddings


#------------------------------------------------------------------------
# ES임베딩 쿼리 벡터
# -쿼리 임베딩 벡터를 구하고, es에 접속해서 rfile_text 뽑아냄
#------------------------------------------------------------------------
def es_embed_query(query:str):
    assert query, f'query is empty'
    
    # 1.elasticsearch 접속
    es = Elasticsearch(ES_URL)  

    # 임베딩 구함.
    start_time = time.time()
    embed_query = embedding([query])[0]
    
    #print(len(embed_query))
    
    # 쿼리 구성
    script_query = {
        "script_score":{
            "query":{
                "match_all": {}},
            "script":{
                "source": "cosineSimilarity(params.query_vector, doc['vector1']) + 1.0",  # 뒤에 1.0 은 코사인유사도 측정된 값 + 1.0을 더해준 출력이 나옴
                "params": {"query_vector": embed_query}
            }
        }
    }
    
    #print(script_query)

    # 실제 ES로 검색 쿼리 날림
    start_search_time = time.time()
    response = es.search(
        index=ES_INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source":{"includes": ["rfile_name", "rfile_text"]}
        }
    )
    end_time = time.time() - start_time
    print("*ES 검색시간: {:.2f} ms".format(end_time * 1000)) 

    count = 0
    docs = []
    for hit in response["hits"]["hits"]: 
        doc = {}  #dict 선언
        doc['rfile_name'] = hit["_source"]["rfile_name"]      # contextid 담음
        doc['rfile_text'] = hit["_source"]["rfile_text"]      # text 담음.
        doc['score'] = hit["_score"]
        docs.append(doc) 
            
    return docs
#------------------------------------------------------------------------


In [13]:
from transformers import GenerationConfig

#--------------------------------------------
# prompt 생성
#--------------------------------------------
def make_prompt(docs, query)->str:
     # prompt 구성
    context:str = ''

    for doc in docs:
        score = doc['score']
        if score > MIN_SCORE:
            rfile_text = doc['rfile_text']
            if rfile_text:
                context += rfile_text + '\n\n'
                
    if context:
        prompt = PROMPT_DICT['prompt_context'].format(query=query, context=context)
    else:
        prompt = PROMPT_DICT['prompt_no_context'].format(query=query)
                
    # KoAlpaca 프롬프트
    #prompt = f"### 질문: {query}\n질문에 대해 아래 내용을 바탕으로 간략히 답변해 주세요\n\n### 문맥: {context}\n\n### 답변:" if context else f"### 질문: {query}\n질문에 대해 간략히 답변해 주세요\n\n### 답변:"
    
    # llama 프롬프트
    #prompt = f"아래는 작업을 설명하는 명령어입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### Instruction: {context}\n{query} ### Response:"

    #print(prompt)
    #print()
    return prompt
    
#-----------------------------------------
# sLLM 이용한 text 생성
#-----------------------------------------
def generate_text_sLLM(prompt):
    
    max_new_tokens = 256
    eos_str = tokenizer.decode(tokenizer.eos_token_id)
    start_time = time.time()

    #prompt = query
    #prompt = f"### 질문: {input_text}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {input_text}\n\n### 답변:"
    #prompt = f"### 질문 : 간략히 답변해줘.{query}\r\n###답변:"
    #prompt = f"### 질문: {query}\n\n### 답변:"

    #print(prompt)

    # config 설정
    generation_config = GenerationConfig(
        temperature=0.5,
        #top_p=0.75,
        #top_k=40,
        #num_beams=1,
        bos_token_id=tokenizer.bos_token_id,  # 시작토큰 
        eos_token_id=tokenizer.eos_token_id,  # end 토큰
        pad_token_id=tokenizer.pad_token_id   # padding 토큰
    )

    # 프롬프트 tokenizer 
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    input_ids = inputs["input_ids"]
    #print(input_ids)

    # Without streaming
    # generate 처리
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=False,
            max_new_tokens=max_new_tokens,
        )

    # 출력
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)

    end_time = time.time() - start_time
    print("*Text생성시간: {:.2f} ms\n".format(end_time * 1000)) 
    #print(output.replace(eos_str, ''))
    #print()
    return output.replace(eos_str, '')

#-----------------------------------------
# GPT를 이용한 text 생성
#-----------------------------------------
import openai

# key 지정
openai.api_key = "sk-xxxx"

# 모델 - GPT 3.5 Turbo 지정
# => 모델 목록은 : https://platform.openai.com/docs/models/gpt-4 참조
model = "gpt-3.5-turbo"#"gpt-4"#"gpt-3.5-turbo" #gpt-4-0314
'''
ROLE_PROMPT = "아래 내용을 요약해서 질문에 알맞게 간략히 답변해주세요"

# 메시지 설정
MESSAGES = [
            {"role": "system", "content": "You are a helful assistant."},
            #{"role": "system", "content": "질문에 대해 요약해줘"},
            #{"role": "user", "content" : "How are you?"},
            #{"role": "assistant", "content" : "I am doing well"},
            {"role": "user", "content": ROLE_PROMPT}
        ]
'''
MESSAGES:list = []
#-----------------------------------------
# GPT를 이용한 text 생성
#-----------------------------------------
def generate_text_GPT(prompt, messages):
    
    print(f'len(messages):{len(messages)}') 
    print()
    
    #-------------------------------------------------------
    # *** gpt에 메시지는 계속 대화 내용이 유지가 되므로, 비용이 발생함.
    # 따라서 최근 2개 대화만 유지함.
    #if len(messages) >= 2:
    #    messages = messages[len(messages)-2:]  # 최근 2개의 대화만 가져오기
    messages = []  # 무조건 최근대화 초기화
     #-------------------------------------------------------
        
    # 사용자 메시지 추가
    messages.append( {"role": "user", "content": prompt})
    print(messages)

    # ChatGPT-API 호출하기
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=500, # 토큰 수 
        temperature=1,  # temperature 0~2 범위 : 작을수록 정형화된 답변, 클수록 유연한 답변(2는 엉뚱한 답변을 하므로, 1.5정도가 좋은것 같음=기본값은=1)
        top_p=0.1 # 기본값은 1 (0.1이라고 하면 10% 토큰들에서 출력 토큰들을 선택한다는 의미)
    )

    print(response)
    print()
    answer = response['choices'][0]['message']['content']
    return answer


#------------------------------------------------------------------
# 구글 bard를 이용한 text 생성
#
# 세션키를 이용하여 구글 bard 테스트 예제
# 출처 : https://github.com/dsdanielpark/Bard-API
#
# token 값얻기
# https://bard.google.com/ 방문
# 콘솔용 F12
# 세션: 애플리케이션 → 쿠키 → 쿠키 값을 복사합니다 __Secure-1PSID.
# -> 참고로 반드시 뒤에 .으로 끝나고 .포함해서 길이가 72자임.
#------------------------------------------------------------------

from bardapi import Bard

token = 'XwhPmzcV-xxxxx.'

def generate_text_BARD(prompt):
    
    bard = Bard(token=token,timeout=30) # Set timeout in seconds
    answer = bard.get_answer(prompt)['content']
    return answer

In [11]:
# 쿼리 입력후 테스트
def run_query_loop():
    while True:
        try:
            handle_query()
        except KeyboardInterrupt:
            return

def handle_query():
    # 쿼리는 일반 쿼리인 경우 '일반##제주도 면적은 얼마?'
    #query = '일반##제주도 면적은 얼마?' 
    query = input("질문을 입력하세요")
    
    query_split = query.split('##')
    prefix = query_split[0]
    #print(f'prefix: {prefix}')
    
    if prefix == '일반':
        query1 = query_split[1]
        prompt = make_prompt(docs='', query=query1)
    else:
        query1 = query
        docs = es_embed_query(query1)
        print(docs)
        print()
        prompt = make_prompt(docs=docs, query=query1)
        
    if LLM_MODEL_TYPE == 0:     # 0=아래 지정한 sLLM 모델, 1=GPT 모델, 2=BARD 모델
        print(generate_text_sLLM(prompt))
    elif LLM_MODEL_TYPE == 1:     # 0=아래 지정한 sLLM 모델, 1=GPT 모델, 2=BARD 모델:
        print(generate_text_GPT(prompt=prompt, messages=MESSAGES))
    else:
        print(generate_text_BARD(prompt))
        
    print()
              

In [14]:
run_query_loop()

질문을 입력하세요 1일 출장시 숙박비는 얼마인가요


/tmp/ipykernel_17410/2780233136.py:46: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es.search(


*ES 검색시간: 31.32 ms
[{'rfile_name': 44, 'rfile_text': '출장 여비 규정\n별표1 국내출장여비 \n임원 이상이면, 철도 운임은 실비(KTX특실), 선박운임은 실비(1등급), 항공운임은 실비(일반실), 자동차 운임은 실비로 지원함.\n임원 이상이면, 1일비는 20,000원 지원함.\n임원 이상이면, 숙박비는 1박당 실비로 지원함.\n임원 이상이면, 1일 식비는 25,000원 지원함.', 'score': 1.5665728}, {'rfile_name': 44, 'rfile_text': '출장 여비 규정\n4. 지급기준\n4.1. 일수 계산 시, 기산은 오전 0시부터로 하며 1일 미만의 단수는 1일로 계산한다.\n숙박비는 숙박일수에 따라 계산하며, 기산은 오전 0시를 경과함으로써 1박으로 하며, 숙박이 다른 출장목적지로의 이동 중 교통편 내에서 이루어졌을 경우 여행 목적지의 해당 숙박비만을 지급한다.', 'score': 1.5133901}, {'rfile_name': 44, 'rfile_text': '출장 여비 규정\n6. 일비, 숙박비 및 식비\n6.1. 일비, 숙박비, 식비의 지급\n6.1.1. 국내 여행자의 일비, 숙박비 및 식비는 별표 1에 따라 지급하고, 국외 여행자의 경우는 별표2에 따라 지급한다. \n다만, 부득이한 사유로 숙박비의 상한을 초과하는 여비 지출이 필요한 경우, 국내 여행의 경우에는 숙박비 상한액의 10분의 3을, 국외 여행의 경우 2분의 1을 넘지 아니하는 범위에서 여비를 추가로 지급할 수 있다. \n이 경우, 국내 여행의 경우, 본부장의 사전 승인을 득해야 하고, 국외 여행의 경우, 대표이사의 사전 승인을 득해야 한다.\n6.1.2. 일비는 여행일수에 따라 지급하되, 법인차량을 이용하는 경우에는 일비의 2분의 1을 지급한다.\n6.1.3. 식비는 여행일수에 따라 지급한다. 다만, 수로여행과 항공여행에 는 따로 식비가 필요한 경우에만 식비를 지급한다.', 'score': 1.4632416}]

1일 출

질문을 입력하세요 1일 식비는 얼마인가요?


*ES 검색시간: 42.43 ms
[{'rfile_name': 44, 'rfile_text': '출장 여비 규정\n6. 일비, 숙박비 및 식비\n6.1. 일비, 숙박비, 식비의 지급\n6.1.1. 국내 여행자의 일비, 숙박비 및 식비는 별표 1에 따라 지급하고, 국외 여행자의 경우는 별표2에 따라 지급한다. \n다만, 부득이한 사유로 숙박비의 상한을 초과하는 여비 지출이 필요한 경우, 국내 여행의 경우에는 숙박비 상한액의 10분의 3을, 국외 여행의 경우 2분의 1을 넘지 아니하는 범위에서 여비를 추가로 지급할 수 있다. \n이 경우, 국내 여행의 경우, 본부장의 사전 승인을 득해야 하고, 국외 여행의 경우, 대표이사의 사전 승인을 득해야 한다.\n6.1.2. 일비는 여행일수에 따라 지급하되, 법인차량을 이용하는 경우에는 일비의 2분의 1을 지급한다.\n6.1.3. 식비는 여행일수에 따라 지급한다. 다만, 수로여행과 항공여행에 는 따로 식비가 필요한 경우에만 식비를 지급한다.', 'score': 1.380271}, {'rfile_name': 6, 'rfile_text': '급여규정\n4. 급여조정\n급여의 인상은 년 1회 1월 1일부로 실시함을 원칙으로 한다.', 'score': 1.3736984}, {'rfile_name': 45, 'rfile_text': '급여규정\n4. 급여조정\n급여의 인상은 년 1회 1월 1일부로 실시함을 원칙으로 한다.', 'score': 1.3736984}]

1일 식비는 국내 출장의 경우 1만 5천원, 국외 출장의 경우 2만 원입니다. 다만, 부득이한 사유로 숙박비의 상한을 초과하는 여비 지출이 필요한 경우, 국내 여행의 경우에는 숙박비 상한액의 10분의 3을, 국외 여행의 경우 2분의 1을 넘지 아니하는 범위에서 여비를 추가로 지급할 수 있습니다.



...

*ES 검색시간: 44.64 ms
[{'rfile_name': 44, 'rfile_text': '출장 여비 규정\n18. 기타사항\n기타 필요한 사항은 건별로 사장이 결정한다.', 'score': 1.4405224}, {'rfile_name': 101, 'rfile_text': '회사 개요\n회사 주요 제품 \nEAI/ESB 분야:  MI(Midas Integration, GS인증 보유), MTE , BM Integration Foundation(Integration Bus, Websphere MQ 등), Vitria BusinessWare\n보안 분야 : Mpower EZis-C (로컬저장금지 및 문서중앙화), Mpower EZis-V (도면유출방지), Mpower B2B (대외 파일 전송), Mpower 보안파일서버 (정보 자산화 및 협업), Mpower Backup (PC백업), Mpower EZis-I (망분리), Mpower S-filter (서버용 개인정보 검출)\n모니터링/AI 분야: dynaTrace, Catchpoint, RWS Language Weaver\nOpen API 분야 : IBM API Connect', 'score': 1.4044698}, {'rfile_name': 14, 'rfile_text': '유형자산 관리 지침\n5. 모니터의 구매\n모니터는 아래 품목 및 규격으로 사용자가 신청, 부서의 결재를 득하여 경영지원본부에서 구매한다.\n- DELL 24인치-권고모델은 DELL P2422H(권고 모델 이외의 제품을 구매할 경우 사업부장 승인 필요)\n- 모니터 제품 선정은 기술 발전 상황 등을 고려하여 필요 시 조정될 수 있다. 기술부서에서 조정의 필요성을 판단, 제품을 선정하고 이를 관리규정에 반영한다.', 'score': 1.3894854}]

회사의 주요 제품은 다음과 같습니다.

* EAI/ESB 분야: MI(Midas Integration, GS인증 보유), MTE , BM Integration Foundation(Inte

질문을 입력하세요 회사에 설치해도 되는 소프트웨어 종류에는 어떤것들이 있나?


*ES 검색시간: 52.96 ms
[{'rfile_name': 9, 'rfile_text': '소프트웨어 관리 규정\n4. 주관부서 및 임무\n회사가 소유한 모든 소프트웨어의 제반관리는 경영지원팀에서 총괄하며 주관부서의 임무는 다음과 같다.\n4.1 소프트웨어의 구매계획 및 예산편성\n4.2 소프트웨어의 도입평가, 선정, 구매, 보급, 설치, 확인\n4.3 소프트웨어의 사용자 교육, 관리(정기점검)\n4.4 소프트웨어의 유지, 보수계약 관리\n4.5 소프트웨어의 VERSION UPGRADE 추진\n4.6 기타, 소프트웨어 관련 업무', 'score': 1.6530732}, {'rfile_name': 8, 'rfile_text': '㈜모코엠시스 보안관리규정\n제7조 (일반적인 소프트웨어 정책)\n공개 소프트웨어 : 설치 책임이 있는 전산 책임자가 저자/소스의 무결성에 대해 확신하는 경우에 한해 설치/사용할 수 있다.\n업무용 소프트웨어 \n업무용 소프트웨어란 OS, Office, 백신, 개발용TOOL , 회계인사 System, KMS 등을 말한다.\n라이센스를 받지 않은 소프트웨어를 사용해서는 안 된다.\n회사의 정책상, 업무의 특성상 상용 소프트웨어의 사용을 제한 할 수 있다.\n이는 미리 업무 책임자와의 협의에 의해 진행되어야 한다.\n회사의 업무에 필요한 소프트웨어의 구매에 대하여 전산 책임자는 업무 책임자와의 협의 하에 구매하여야 한다.\n기타 업무용으로 개발된 소프트웨어의 접근 권한을 업무 범위에 따라 시스템별로 접근을 제한할 수 있다. ', 'score': 1.5971236}, {'rfile_name': 50, 'rfile_text': '㈜모코엠시스 보안관리규정\n제7조 (일반적인 소프트웨어 정책)\n공개 소프트웨어 : 설치 책임이 있는 전산 책임자가 저자/소스의 무결성에 대해 확신하는 경우에 한해 설치/사용할 수 있다.\n업무용 소프트웨어 \n업무용 소프트웨어란 OS, Office, 백신, 개발용TOOL , 회계인사 System, KMS 등을 말한다

질문을 입력하세요 개인 보호법이라는게 뭔가요?


*ES 검색시간: 40.17 ms
[{'rfile_name': 16, 'rfile_text': '개인정보 내부 관리계획\n제7조(개인정보 보호책임자의 역할 및 책임) ① 개인정보 보호책임자는 다음 각 호의 업무를 수행한다.\n1. 개인정보 보호 계획의 수립 및 시행\n2. 개인정보 처리 실태 및 관행의 정기적인 조사 및 개선\n3. 개인정보 처리와 관련한 불만의 처리 및 피해 구제\n4. 개인정보 유출 및 오용․남용 방지를 위한 내부통제시스템의 구축\n5. 개인정보 보호 교육 계획의 수립 및 시행\n6. 개인정보파일의 보호 및 관리 감독\n7.「개인정보 보호법」 제30조에 따른 개인정보 처리방침의 수립․변경 및 시행\n8. 개인정보 보호 관련 자료의 관리\n9. 처리 목적이 달성되거나 보유기간이 지난 개인정보의 파기\n② 개인정보 보호책임자는 제1항의 업무를 수행함에 있어서 필요한 경우 개인정보의 처리 현황, 처리 체계 등에 대하여 수시로 조사하거나 관계 당사자로부터 보고를 받을 수 있다.\n③ 개인정보 보호책임자는 개인정보 보호와 관련하여 이 법 및 다른 관계 법령의 위반 사실을 알게 된 경우에는 즉시 개선조치를 하여야 하며, 필요하면 대표이사에게 개선조치를 보고하여야 한다. ', 'score': 1.5931453}, {'rfile_name': 51, 'rfile_text': '\n개인정보 내부 관리계획\n제7조(개인정보 보호책임자의 역할 및 책임) ① 개인정보 보호책임자는 다음 각 호의 업무를 수행한다.\n1. 개인정보 보호 계획의 수립 및 시행\n2. 개인정보 처리 실태 및 관행의 정기적인 조사 및 개선\n3. 개인정보 처리와 관련한 불만의 처리 및 피해 구제\n4. 개인정보 유출 및 오용․남용 방지를 위한 내부통제시스템의 구축\n5. 개인정보 보호 교육 계획의 수립 및 시행\n6. 개인정보파일의 보호 및 관리 감독\n7.「개인정보 보호법」 제30조에 따른 개인정보 처리방침의 수립․변경 및 시행\n8. 개인정보 보호 관련 자료의 관리\n9. 처리 목적이 